# Processing Big Data - Data Ingestion
© Explore Data Science Academy

## Honour Code
I {**ONIDAJO**, **WALE-OLAITAN**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.



## Context 

To work constructively with any dataset, one needs to create an ingestion profile to make sure that the data at the source can be readily consumed. For this section of the predict, as the Data Engineer in the team, you will be required to design and implement the ingestion process. For the purposes of the project the AWS cloud storage service, namely, the S3 bucket service will act as your data source. All the data required can be found [here](https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip).

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/DataIngestion.jpg"
     alt="Data Ingestion"
     style="float: center; padding-bottom=0.5em"
     width=40%/>
     <p><em>Figure 1. Data Ingestion</em></p>
</div>

Your manager, Gnissecorp Atadgib, knowing very well that you've recently completed your Data Engineering qualification, asks you to make use of Apache Spark for the ingestion as well as the rest of the project. His rationale being, that stock market data is generated every day and is quite time-sensitive and would require scalability when deploying to a production environment. 

## Dataset - US Nasdaq




<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/Nasdaq.png"
     alt="Nasdaq"
     style="float: center; padding-bottom=0.5em"
     width=50%/>
     <p><em>Figure 2. Nasdaq</em></p>
</div>

The data that you will be working with is a historical snapshot of market data taken from the Nasdaq electronic market. This dataset contains historical daily prices for all tickers currently trading on Nasdaq. The up-to-date list can be found on their [website](https://www.nasdaq.com/)


The provided data contains price data dating back from 02 January 1962 up until 01 April 2020. The data found in the S3 bucket has been stored in the following structure:

```
     stocks/<Year>/<Month>/<Day>/stocks.csv
```
Each CSV file for every trading day contains the following details:
- **Date** - specifies trading date
- **Open** - opening price
- **High** - maximum price during the day
- **Low** - minimum price during the day
- **Close** - close price adjusted for splits
- **Adj Close** - close price adjusted for both dividends and splits
- **Volume** - the number of shares that changed hands during a given day

## Basic initialisation
To get you started, let's import some basic Python libraries as well as Spark modules and functions.

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

Remember that we need a `SparkContext` and `SparkSession` to interface with Spark.
We will mostly be using the `SparkContext` to interact with RDDs and the `SparkSession` to interface with Python objects.

> ℹ️ **Instructions** ℹ️
>
>Initialise a new **Spark Context** and **Session** that you will use to interface with Spark.

In [65]:
#TODO: Write your code here
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Investigate dataset schema
At this point, it is enough to read in a single file to ascertain the data structure. You will be required to use the information obtained from the small subset to create a data schema. This data schema will be used when reading the entire dataset using Spark.

> ℹ️ **Instructions** ℹ️
>
>Make use of Pandas to read in a single file and investigate the plausible data types to be used when creating a Spark data schema. 
>
>*You may use as many coding cells as necessary.*

In [66]:
#TODO: Write your code here
pd_df = pd.read_csv('../../../../Desktop/Explore/Processing Big Data/stocks/1962/01/02/stocks.csv')

In [67]:
pd_df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
stock         object
dtype: object

In [68]:
pd_df.head()

Date      Open      High       Low     Close  Adj Close    Volume  \
0  1962-01-02  6.532155  6.556185  6.532155  6.532155   1.536658   55900.0   
1  1962-01-02  6.125844  6.160982  6.125844  6.125844   1.414651   59700.0   
2  1962-01-02  0.837449  0.837449  0.823045  0.823045   0.145748  352200.0   
3  1962-01-02  1.604167  1.619792  1.588542  1.604167   0.136957  163200.0   
4  1962-01-02  0.000000  3.296131  3.244048  3.296131   0.051993  105600.0   

  stock  
0    AA  
1  ARNC  
2    BA  
3   CAT  
4   CVX

## Read CSV files

When working with big data, it is often not tenable to keep processing an entire data batch when you are in the process of development - this can be quite time-consuming. If the data is uniform, it is sufficient to work with a smaller subset to create basic functionality. Your manager has identified the year **1962** to perform the initial testing for data ingestion. 

> ℹ️ **Instructions** ℹ️
>
>Read in the data for **1962** using a data schema that purely uses string data types. You will be required to convert to the appropriate data types at a later stage.
>
>*You may use as many coding cells as necessary.*

In [69]:
schema = StructType([StructField('Date', StringType(), True),
                     StructField('Open', StringType(), True),
                     StructField('High', StringType(), True),
                     StructField('Low', StringType(), True),
                     StructField('Close', StringType(), True),
                     StructField('Adj Close', StringType(), True),
                     StructField('Volume', StringType(), True),
                     StructField('stock', StringType(), True)])

In [70]:
#TODO: Write your code here
df = spark.read.csv('../../../../Desktop/Explore/Processing Big Data/Stocks/1962/*/*/', header=True, schema=schema)

In [71]:
df.count()

5106

In [72]:
from datetime import date
filter_date = date(1962, 1, 2)
df.where(F.col('Date') == filter_date).show()

+----------+-------------------+-------------------+-------------------+-------------------+--------------------+---------+-----+
|      Date|               Open|               High|                Low|              Close|           Adj Close|   Volume|stock|
+----------+-------------------+-------------------+-------------------+-------------------+--------------------+---------+-----+
|1962-01-02| 6.5321550369262695|  6.556184768676758| 6.5321550369262695| 6.5321550369262695|  1.5366575717926023|  55900.0|   AA|
|1962-01-02|  6.125843524932861|  6.160982131958008|  6.125843524932861|  6.125843524932861|  1.4146506786346436|  59700.0| ARNC|
|1962-01-02| 0.8374485373497009| 0.8374485373497009| 0.8230452537536621| 0.8230452537536621|  0.1457476019859314| 352200.0|   BA|
|1962-01-02| 1.6041666269302368| 1.6197916269302368| 1.5885416269302368| 1.6041666269302368| 0.13695742189884186| 163200.0|  CAT|
|1962-01-02|                0.0| 3.2961308956146236| 3.2440476417541504| 3.296130895614623

In [73]:
df.show()

+----------+------------------+------------------+------------------+------------------+--------------------+---------+-----+
|      Date|              Open|              High|               Low|             Close|           Adj Close|   Volume|stock|
+----------+------------------+------------------+------------------+------------------+--------------------+---------+-----+
|1962-02-19| 5.839290142059326| 5.907374858856201| 5.839290142059326| 5.863319873809815|  1.3863292932510376|  29900.0|   AA|
|1962-02-19| 5.481634140014648|5.5284857749938965| 5.481634140014648| 5.516772747039795|  1.2804527282714844|  32000.0| ARNC|
|1962-02-19|0.9074074029922484|0.9156378507614136|0.8991769552230835| 0.903292179107666|  0.1614154428243637| 619400.0|   BA|
|1962-02-19|1.6770833730697632|1.6927083730697632|1.6614583730697632|1.6770833730697632|  0.1440587043762207| 170400.0|  CAT|
|1962-02-19|               0.0|3.5788691043853764|              20.0| 3.549107074737549|  0.0565012246370315| 273600.0

## Update column names
To make the data easier to work with, you will need to make a few changes:
1. Column headers should all be in lowercase; and
2. Whitespaces should be replaced with underscores.


> ℹ️ **Instructions** ℹ️
>
>Make sure that the column headers are all in lowercase and that any whitespaces are replaced with underscores.
>
>*You may use as many coding cells as necessary.*

In [74]:
#TODO: Write your code here
for column in df.columns:
    df = df.withColumnRenamed(column, '_'.join(column.split()).lower().replace(' ', ''))
# Display our results.
df.show(5)

+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|      date|              open|              high|               low|             close|         adj_close|  volume|stock|
+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|1962-02-19| 5.839290142059326| 5.907374858856201| 5.839290142059326| 5.863319873809815|1.3863292932510376| 29900.0|   AA|
|1962-02-19| 5.481634140014648|5.5284857749938965| 5.481634140014648| 5.516772747039795|1.2804527282714844| 32000.0| ARNC|
|1962-02-19|0.9074074029922484|0.9156378507614136|0.8991769552230835| 0.903292179107666|0.1614154428243637|619400.0|   BA|
|1962-02-19|1.6770833730697632|1.6927083730697632|1.6614583730697632|1.6770833730697632|0.1440587043762207|170400.0|  CAT|
|1962-02-19|               0.0|3.5788691043853764|              20.0| 3.549107074737549|0.0565012246370315|273600.0|  CVX|
+----------+----

## Null Values
Null values often represent missing pieces of data. It is always good to know where your null values lie - so you can quickly identify and remedy any issues stemming from these.

> ℹ️ **Instructions** ℹ️
>
>Write code to count the number of null values found in each column.
>
>*You may use as many coding cells as necessary.*

In [75]:
#TODO: Write your code here
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show(5)

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 22|    0|        0|    21|    0|
+----+----+----+---+-----+---------+------+-----+



In [76]:
df.select([F.count(F.when(F.isnan(c), c)).alias(c) for c in df.columns]).show(5)

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0|  0|    0|        0|     0|    0|
+----+----+----+---+-----+---------+------+-----+



## Data type conversion - The final data schema

Now that we have identified the number of missing values in the data set, we'll move on to convert our data schema to the required data types. 

> ℹ️ **Instructions** ℹ️
>
>Use typecasting to convert the string data types in your current data schema to more appropriate data types.
>
>*You may use as many coding cells as necessary.*

In [77]:
df_one=df.withColumn('open', F.col('open').cast(FloatType()))

In [78]:
df_one=df_one.withColumn('high', F.col('high').cast(FloatType()))

In [79]:
df_one=df_one.withColumn('low', F.col('low').cast(FloatType()))

In [80]:
df_one=df_one.withColumn('close', F.col('close').cast(FloatType()))

In [81]:
#TODO: Write your code here
df_one=df_one.withColumn('adj_close', F.col('adj_close').cast(FloatType()))

In [82]:
df_one=df_one.withColumn('volume', F.col('volume').cast(IntegerType()))

In [83]:
#df_one=df_one.withColumn('date', F.col('date').cast(TimestampType()))

In [84]:
df_one.printSchema()

root
 |-- date: string (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- adj_close: float (nullable = true)
 |-- volume: integer (nullable = true)
 |-- stock: string (nullable = true)



## Consolidate missing values
We have to check if the data type conversion above was done correctly.
If the casting was not successful, a null value gets inserted into the dataframe. You can thus check for successful conversion by determining if any null values are included in the resulting dataframe.


> ℹ️ **Instructions** ℹ️
>
>Write code to compare the number of invalid entries (nulls) pre-conversion and post-conversion.
>
>*You may use as many coding cells as necessary.*

In [85]:
table = ["open","high", "low", "close", "adj_close", "volume"]
for col in table:
    # Again, we are using the isnull() function to find values that are null.
    # We use the where() method to only select records that match our conditions.
    # Our conditions are where the casted column is null and the raw df is not null (we use the '~' operator which signifies not).
    # Similar to how we aggregate using count and return each DataFrame to standard out.
    df_one.where(F.isnull(df_one[col])  & ~F.isnull(df[col])).groupby(col, df_one[col]).count().show()

+----+----+-----+
|open|open|count|
+----+----+-----+
+----+----+-----+

+----+----+-----+
|high|high|count|
+----+----+-----+
+----+----+-----+

+----+----+-----+
| low| low|count|
+----+----+-----+
|null|null|   20|
+----+----+-----+

+-----+-----+-----+
|close|close|count|
+-----+-----+-----+
+-----+-----+-----+

+---------+---------+-----+
|adj_close|adj_close|count|
+---------+---------+-----+
|     null|     null|   21|
+---------+---------+-----+

+------+------+-----+
|volume|volume|count|
+------+------+-----+
+------+------+-----+



In [86]:
test_df = df
failures = {}
table = ["open","high", "low", "close", "adj_close"]
for col in table:
    # Count the number of nulls before type casting, similar to the above counts.
    before_count = test_df.select(F.count(F.when(F.isnull(col), col)).alias(col)).collect() 
    
    # Cast to Float type, creating a new column using the withColumn() method.
    # We use the cast() method to cast and use the FloatType defined in Spark.
    test_df = test_df.withColumn(f'test_type_{col}', F.col(col).cast(FloatType())) 
    
    # Count the number of nulls after type casting, similar to the above counts.
    after_count = test_df.select(F.count(F.when(F.isnull(f'test_type_{col}'), f'test_type_{col}')).alias(f'test_type_{col}')).collect() 
    
    # Get the difference between before and after.
    failures[col] = after_count[0][0] - before_count[0][0] 

In [87]:
failures

{'open': 0, 'high': 0, 'low': 20, 'close': 0, 'adj_close': 21}

In [88]:
#TODO: Write your code here
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show(5)

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 22|    0|        0|    21|    0|
+----+----+----+---+-----+---------+------+-----+



In [89]:
#TODO: Write your code here
df_one.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show(5)

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 42|    0|       21|    21|    0|
+----+----+----+---+-----+---------+------+-----+



Here you should be able to see if any of your casts went wrong. 
Do not attempt to correct any missing values at this point. This will be dealt with in later sections of the predict.

## Generate parquet files
When writing in Spark, we typically use parquet format. This format allows parallel writing using Spark's optimisation while maintaining other useful things like metadata.

When writing, it is good to make sure that the data is sufficiently partitioned. 

Generally, data should be partitioned with one partition for every 200MB of data, but this also depends on the size of your cluster and executors. 


### Check the size of the dataframe before partitioning

In [90]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer

In [91]:
rdd = df.rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
obj = rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)
size = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(obj)
size_MB = size/1000000
partitions = max(int(size_MB/200), 2)
print(f'The dataframe is {size_MB} MB')

The dataframe is 51.099952 MB


In [92]:
partitions

2

### Write parquet files to the local directory
> ℹ️ **Instructions** ℹ️
>
> Use the **coalesce** function and the number of **partitions** derived above to write parquet files to your local directory 
>
>*You may use as many coding cells as necessary.*

In [93]:
#TODO: Write your code here
df_one.coalesce(partitions).write.format("parquet").save("../transformed_data")